<a href="https://colab.research.google.com/github/Noxybot/tt_21/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [294]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.optimizers import *

SEED = 228

tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

GEO_CLUSTERS = 300
COMPANY_UNITS = 10
ACTIVATION_FUN = "relu"


In [246]:
train_data = pd.read_csv("train.csv")
train_data = train_data.drop(train_data.columns[0], axis=1)

vc = train_data["company"].value_counts()
to_remove = vc[vc <= 10].index

train_data.loc[train_data.company.isin(to_remove), "company"] = -1

test_data = pd.read_csv("test.csv")
test_data = test_data.drop(test_data.columns[0], axis=1)
test_data.loc[test_data.company.isin(to_remove), "company"] = -1
test_data.loc[~test_data.company.isin(train_data.company), "company"] = -1
test_data

,latitude,longitude,company,is_local,type,fin_1,fin_2,fin_3,fin_4,target
0,39.284310,-76.735350,-1,1,1,113033.389907,0.000000,270.906219,31222.780176,0
1,36.758509,-76.344861,11208,0,0,-87239.590275,73759.387510,759.194862,237587.544996,1
2,43.402802,-75.217100,-1,1,4,-122084.498620,15528.109943,1039.655934,29612.346982,0
3,41.871160,-87.848570,-1,0,1,87355.127256,0.000000,214.594205,163526.475818,1
4,42.161296,-88.129184,11515,0,3,33014.437946,0.000000,379.819724,67499.397999,1
...,...,...,...,...,...,...,...,...,...,...
39995,35.241600,-80.983740,-1,0,3,194648.093228,0.000000,95.701485,37584.380280,1
39996,35.241730,-80.983750,11849,0,3,-53202.007008,9641.082811,653.516413,189604.463353,0
39997,35.290596,-80.756953,3987,0,3,-102398.568309,47105.929570,863.135873,26231.880089,0
39998,35.204460,-80.720190,12287,0,0,32066.188506,6659.705140,363.861756,116288.412218,1


In [247]:
def preprocess (dataframe, geo_label, company_label):
  dataframe['geo_cluster_label'] = geo_label.predict(dataframe[dataframe.columns[0:2]])
  dataframe = dataframe.drop('latitude', axis=1)
  dataframe = dataframe.drop('longitude', axis=1)
  first_X = pd.concat([dataframe["is_local"], dataframe.iloc[:, 3:7]], axis=1).to_numpy()
  second_X = company_label.transform(dataframe["company"].to_numpy())
  third_X = dataframe["type"].to_numpy()
  fourth_X = dataframe["geo_cluster_label"].to_numpy()

  out_Y = dataframe["target"].to_numpy()
  return first_X, second_X, third_X, fourth_X, out_Y



In [248]:
kmeans = KMeans(n_clusters = GEO_CLUSTERS, init ='k-means++')
le = LabelEncoder()

kmeans.fit(train_data[train_data.columns[0:2]])
le.fit(train_data["company"].to_numpy())

first_X_train, second_X_train, third_X_train, fourth_X_train, out_Y_train = preprocess(train_data, kmeans, le)
first_X_test, second_X_test, third_X_test, fourth_X_test, out_Y_test = preprocess(test_data, kmeans, le)


In [295]:
input_first_branch = Input(shape=(5, ), name="first")
first = Dense(32, activation=ACTIVATION_FUN)(input_first_branch)
first = BatchNormalization()(first)
first = Dense(32, activation=ACTIVATION_FUN)(first)
first = BatchNormalization()(first)
first = Model(inputs=input_first_branch, outputs=first)

input_second_branch = Input(shape=(1,), name="second")
second = Embedding(second_X_train.max() + 1, output_dim=128)(input_second_branch)
second = Flatten()(second)
second = Dense(32, activation=ACTIVATION_FUN)(second)
second = Model(inputs=input_second_branch, outputs=second)

input_third_branch = Input(shape=(1,), name="third")
third = Embedding(third_X_train.max() + 1, output_dim=16)(input_third_branch)
third = Flatten()(third)
third = Dense(16, activation=ACTIVATION_FUN)(third)
third = Model(inputs=input_third_branch, outputs=third)


input_fourth_branch = Input(shape=(1,), name="fourth")
fourth = Embedding(fourth_X_train.max() + 1, output_dim=128)(input_fourth_branch)
fourth = Flatten()(fourth)
fourth = Dense(32, activation=ACTIVATION_FUN)(fourth)
fourth = Model(inputs=input_fourth_branch, outputs=fourth)

concat = Concatenate()([first.output, second.output, third.output, fourth.output])

concat = Dense(64, activation=ACTIVATION_FUN)(concat)
concat = BatchNormalization()(concat)
concat = Dense(64, activation=ACTIVATION_FUN)(concat)
concat = BatchNormalization()(concat)
output = Dense(1, name="output", activation='sigmoid')(concat)

my_model = Model(inputs=[first.input, second.input, third.input, fourth.input], outputs=output)
opt = Adam(learning_rate=0.01)
my_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
my_model.fit(epochs=10, batch_size=128, shuffle=True, validation_split=0.1, x={"first": first_X_train, "second": second_X_train, "third": third_X_train, "fourth": fourth_X_train }, y=out_Y_train)


Epoch 1/10
1125/1125 [==============================] - 12s 9ms/step - loss: 0.4468 - accuracy: 0.7682 - val_loss: 0.4417 - val_accuracy: 0.7698
Epoch 2/10
1125/1125 [==============================] - 10s 9ms/step - loss: 0.4380 - accuracy: 0.7703 - val_loss: 0.4406 - val_accuracy: 0.7733
Epoch 3/10
1125/1125 [==============================] - 9s 8ms/step - loss: 0.4343 - accuracy: 0.7707 - val_loss: 0.4411 - val_accuracy: 0.7673
Epoch 4/10
1125/1125 [==============================] - 10s 9ms/step - loss: 0.4317 - accuracy: 0.7712 - val_loss: 0.4516 - val_accuracy: 0.7677
Epoch 5/10
1125/1125 [==============================] - 9s 8ms/step - loss: 0.4295 - accuracy: 0.7746 - val_loss: 0.4409 - val_accuracy: 0.7701
Epoch 6/10
1125/1125 [==============================] - 9s 8ms/step - loss: 0.4268 - accuracy: 0.7747 - val_loss: 0.4438 - val_accuracy: 0.7641
Epoch 7/10
1125/1125 [==============================] - 9s 8ms/step - loss: 0.4255 - accuracy: 0.7744 - val_loss: 0.4463 - val_accura

In [296]:
my_model.summary()

Model: "model_279"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 first (InputLayer)             [(None, 5)]          0           []                               
                                                                                                  
 dense_385 (Dense)              (None, 32)           192         ['first[0][0]']                  
                                                                                                  
 second (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 third (InputLayer)             [(None, 1)]          0           []                               
                                                                                          

In [297]:
my_model.evaluate(x={"first": first_X_test, "second": second_X_test, "third": third_X_test, "fourth": fourth_X_test }, y=out_Y_test)


1250/1250 [==============================] - 4s 3ms/step - loss: 0.4501 - accuracy: 0.7699


[0.45010289549827576, 0.7699249982833862]